In [1]:
import numpy as np
import pandas as pd
import cv2 

dataset_path = 'fer2013.csv'
image_size=(48,48)

def load_fer():
        data = pd.read_csv(dataset_path)
        pixels = data['pixels'].tolist()
        width, height = 48, 48
        faces = []
        for pixel_sequence in pixels:
            face = [int(pixel) for pixel in pixel_sequence.split(' ')]
            face = np.asarray(face).reshape(width, height)
            face = cv2.resize(face.astype('uint8'),image_size)
            faces.append(face.astype('float32'))
        faces = np.asarray(faces)
        faces = np.expand_dims(faces, -1)
        emotions = pd.get_dummies(data['emotion']).values
        return faces, emotions

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

In [7]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, BatchNormalization, Activation, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.callbacks import CSVLogger, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.layers import Reshape, Permute

from keras.layers import Input, LSTM, Dense, BatchNormalization, Activation, Reshape, Permute
from keras.regularizers import l2

def mini_XCEPTION_LSTM(input_shape, num_classes, l2_regularization=0.01):
    img_input = Input(input_shape)

    # Reshape the input to represent sequences over time
    x = Reshape((input_shape[0] * input_shape[1], input_shape[2]))(img_input)
    x = Permute((2, 1))(x)  # Transpose the input for LSTM

    x = LSTM(64, return_sequences=True)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = LSTM(64)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Dense(num_classes, kernel_regularizer=l2(l2_regularization), activation='softmax')(x)

    model = Model(img_input, x)
    return model



# parameters
batch_size = 32
num_epochs = 20
input_shape = (48, 48, 1)  # Assuming you want to use the same input shape as before
validation_split = .2
verbose = 1
num_classes = 7
patience = 50
base_path = 'models/'

# data generator
# (No changes needed if you are using the same data generator)

# model parameters/compilation
model = mini_XCEPTION_LSTM(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# callbacks
# (No changes needed to callbacks)

# loading dataset
faces, emotions = load_fer()
faces = preprocess_input(faces)
num_samples, num_classes = emotions.shape
xtrain, xtest, ytrain, ytest = train_test_split(faces, emotions, test_size=0.2, shuffle=True)
# ... (previous code remains the same)

# Assuming you have one-hot encoded ytrain and ytest

# Define your callbacks
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = base_path + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint("models/_mini_XCEPTION.{epoch:02d}-{val_accuracy:.2f}.hdf5", monitor='val_accuracy', save_best_only=True, mode='max')

callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

# Now you can fit the model
model.fit(xtrain, ytrain,
          batch_size=batch_size,
          epochs=num_epochs,
          verbose=verbose,
          validation_data=(xtest, ytest),
          callbacks=callbacks)




Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 reshape_3 (Reshape)         (None, 2304, 1)           0         
                                                                 
 permute_3 (Permute)         (None, 1, 2304)           0         
                                                                 
 lstm_7 (LSTM)               (None, 1, 64)             606464    
                                                                 
 batch_normalization_6 (Bat  (None, 1, 64)             256       
 chNormalization)                                                
                                                                 
 activation_6 (Activation)   (None, 1, 64)             0         
                                                           

C:\Users\gargi\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


898/898 [==============================] - 38s 27ms/step - loss: 1.7955 - accuracy: 0.3171 - val_loss: 1.6833 - val_accuracy: 0.3548 - lr: 0.0010
Epoch 2/20
898/898 [==============================] - 19s 21ms/step - loss: 1.6584 - accuracy: 0.3648 - val_loss: 1.6363 - val_accuracy: 0.3697 - lr: 0.0010
Epoch 3/20
898/898 [==============================] - 19s 21ms/step - loss: 1.6063 - accuracy: 0.3796 - val_loss: 1.6060 - val_accuracy: 0.3823 - lr: 0.0010
Epoch 4/20
898/898 [==============================] - 19s 21ms/step - loss: 1.5688 - accuracy: 0.3981 - val_loss: 1.5743 - val_accuracy: 0.3918 - lr: 0.0010
Epoch 5/20
898/898 [==============================] - 19s 22ms/step - loss: 1.5405 - accuracy: 0.4076 - val_loss: 1.5812 - val_accuracy: 0.3888 - lr: 0.0010
Epoch 6/20
898/898 [==============================] - 19s 21ms/step - loss: 1.5164 - accuracy: 0.4216 - val_loss: 1.5670 - val_accuracy: 0.3922 - lr: 0.0010
Epoch 7/20
898/898 [==============================] - 17s 19ms/step -